In [1]:
# gym_exercise_corrector.py

# 1. Instalação das Dependências
# Garantir que todas as bibliotecas necessárias estejam instaladas.
# Adicionamos 'mediapipe' que é essencial para a estimativa de pose.
# O pacote 'tensorflow' por padrão instala a versão otimizada para CPU.

# Removido 'tensorflow-gpu' e 'gdown' já que estamos em CPU e o dataset é baixado com kaggle cli
# A linha abaixo deve ser executada uma vez no terminal ou na célula de notebook para instalar/atualizar
# !pip install tensorflow numpy pandas matplotlib requests scikit-learn imbalanced-learn tqdm kaggle mediapipe opencv-python --upgrade
# !pip install --upgrade h5py Keras

# 2. Configurações Iniciais e Verificação de Hardware
# Importações essenciais para o projeto.
import os
import shutil
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf
from tqdm import tqdm
import zipfile
import json
from pathlib import Path
import subprocess
import time
import requests
import sys # Importado para sys.exit()

# Importar MediaPipe para estimativa de pose
import mediapipe as mp

# Importar bibliotecas para modelos Scikit-learn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib # Para salvar/carregar scalers e modelos Scikit-learn

# Configuração de CPU
# Em um ambiente com apenas CPU, o TensorFlow irá automaticamente utilizar o processador.
print("ℹ️ Nenhuma GPU NVIDIA detectada ou configurada. Todas as operações de Machine Learning utilizarão a CPU.")

# Configurar credenciais do Kaggle
# Isso é crucial para baixar o dataset de exercícios automaticamente.
kaggle_dir = Path.home() / '.kaggle'
kaggle_dir.mkdir(exist_ok=True)

# Você pode obter isso no seu perfil do Kaggle, na seção "API".
kaggle_creds = {
    "username": "SEU_USERNAME_KAGGLE", # Substitua pelo seu username do Kaggle
    "key": "SUA_CHAVE_KAGGLE" # Substitua pela sua chave API do Kaggle
}

# Salvar credenciais no arquivo kaggle.json
with open(kaggle_dir / 'kaggle.json', 'w') as f:
    json.dump(kaggle_creds, f)

# Adicionar ao PATH para que o comando 'kaggle' possa ser executado
# Isso é importante para que o subprocess.run possa encontrar o comando kaggle.
os.environ['PATH'] += os.pathsep + str(Path.home() / '.local' / 'bin')

print("✅ Ambiente configurado. Credenciais do Kaggle salvas e ambiente preparado para CPU.")


# 3. Definição de Funções Auxiliares para Extração de Features (MediaPipe) e Geração de Erros

# Inicializar o MediaPipe Pose para detecção de landmarks.
# Static_image_mode=False é para processamento de vídeo, True para imagens estáticas.
# Min_detection_confidence e min_tracking_confidence são limiares de confiança.
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles # Corrigido aqui, estava mp.solutions.drawing_utils novamente

# Função para calcular o ângulo entre três pontos (landmarks)
def calculate_angle(a, b, c):
    """
    Calcula o ângulo em graus entre três pontos (landmarks).
    O ponto 'b' é o vértice do ângulo.
    Args:
        a (list/tuple): Coordenadas (x, y) ou (x, y, z) do primeiro ponto.
        b (list/tuple): Coordenadas (x, y) ou (x, y, z) do ponto central (vértice).
        c (list/tuple): Coordenadas (x, y) ou (x, y, z) do terceiro ponto.
    Returns:
        float: O ângulo em graus.
    """
    a = np.array(a) # Primeiro ponto
    b = np.array(b) # Ponto do vértice
    c = np.array(c) # Terceiro ponto

    # Vetores formados pelos pontos
    ba = a - b
    bc = c - b

    # Produto escalar e normas dos vetores
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    # Garantir que o valor esteja dentro do domínio de arccos para evitar erros de floating point
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

# Função para extrair features de pose de uma imagem (ângulos e distâncias)
def extract_pose_features(image_path, pose_detector):
    """
    Extrai um vetor de features de pose (ângulos chave do corpo) de uma imagem.
    Args:
        image_path (str): Caminho para a imagem.
        pose_detector (mp.solutions.pose.Pose): Objeto MediaPipe Pose inicializado.
    Returns:
        numpy.ndarray or None: Um vetor numpy de features (ângulos), ou None se nenhuma pose for detectada.
    """
    try:
        # Lendo a imagem
        image = cv2.imread(image_path)
        if image is None:
            # print(f"⚠️ Não foi possível carregar a imagem: {image_path}") # Comentado para evitar poluir o log
            return None

        # Convertendo a imagem de BGR para RGB (MediaPipe espera RGB)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Processando a imagem para estimativa de pose
        results = pose_detector.process(image_rgb)

        # Se houver landmarks de pose detectadas
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Mapeamento das landmarks do MediaPipe para fácil acesso
            # Para o `mp_pose.PoseLandmark` o `.value` é necessário para acessar o índice correto
            
            # Ângulos dos cotovelos (ombro, cotovelo, punho)
            left_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            )
            right_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            )

            # Ângulos dos ombros (quadril, ombro, cotovelo)
            left_shoulder_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            )
            right_shoulder_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            )
            
            # Ângulos dos joelhos (quadril, joelho, tornozelo)
            left_knee_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            )
            right_knee_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            )

            # Ângulos dos quadris (ombro, quadril, joelho) - essencial para postura de tronco
            left_hip_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            )
            right_hip_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            )
            
            # Coletar todas as features em um vetor
            features = [
                left_elbow_angle, right_elbow_angle,
                left_shoulder_angle, right_shoulder_angle,
                left_knee_angle, right_knee_angle,
                left_hip_angle, right_hip_angle
            ]
            
            return np.array(features)
        else:
            return None # Nenhuma pose detectada
    except Exception as e:
        # print(f"⚠️ Erro ao extrair features de {image_path}: {e}") # Comentado para evitar poluir o log
        return None

# Nova função para gerar formas incorretas sinteticamente
def synthesize_incorrect_form(original_features, exercise_type_str):
    """
    Aplica uma perturbação sintética a uma cópia do array de features original
    para simular uma forma incorreta comum para o tipo de exercício especificado.

    Args:
        original_features (np.ndarray): O array de features (ângulos) da pose original (correta).
        exercise_type_str (str): O nome do exercício (ex: 'squat', 'deadlift', 'bicep curl').

    Returns:
        np.ndarray: O array de features modificado para simular a forma incorreta.
    """
    modified_features = original_features.copy()
    
    # Magnitude da perturbação angular (em graus)
    deviation_magnitude = np.random.uniform(15, 35) # Desvio de 15 a 35 graus

    if exercise_type_str == 'squat':
        # Erro comum: Não ir fundo o suficiente (joelhos não dobram o bastante)
        # Aumentar o ângulo do joelho para simular falta de profundidade
        modified_features[4] = np.clip(modified_features[4] + deviation_magnitude, 0, 180) # Left knee
        modified_features[5] = np.clip(modified_features[5] + deviation_magnitude, 0, 180) # Right knee
        
    elif exercise_type_str == 'deadlift':
        # Erro comum: Costas arredondadas (quadril muito baixo ou muito reto)
        # Diminuir o ângulo do quadril para simular um tronco menos inclinado para frente ou mais arredondado
        modified_features[6] = np.clip(modified_features[6] - deviation_magnitude, 0, 180) # Left hip
        modified_features[7] = np.clip(modified_features[7] - deviation_magnitude, 0, 180) # Right hip
        
    elif exercise_type_str == 'bicep curl':
        # Erro comum: Extensão incompleta (não esticar totalmente o braço na parte inferior)
        # Manter o ângulo do cotovelo maior do que deveria estar na extensão máxima
        modified_features[0] = np.clip(modified_features[0] - deviation_magnitude, 0, 180) # Left elbow (make it less straight)
        modified_features[1] = np.clip(modified_features[1] - deviation_magnitude, 0, 180) # Right elbow (make it less straight)
        
    # Para outros exercícios não especificados, o desvio não será aplicado.
    return modified_features

print("✅ Funções de extração de pose, cálculo de ângulos e geração de erros sintéticos prontas.")

# 4. Baixar e Pré-processar o Dataset de Exercícios (Com Geração de Erros)

# Definir o nome do dataset do Kaggle para exercícios
DATASET_SLUG = "hasyimabdillah/workoutexercises-images"
DATASET_NAME = "workout-exercises-images" # Nome da pasta onde será extraído
DATASET_PATH = Path(DATASET_NAME) # Caminho local

# Função para baixar e extrair o dataset do Kaggle
def download_exercise_dataset(dataset_slug, target_path):
    if not target_path.exists():
        print(f"⬇️ Baixando {dataset_slug} do Kaggle...")
        try:
            # Comando Kaggle para baixar e descompactar
            # O subprocess.run precisa que o comando 'kaggle' esteja acessível no PATH do ambiente
            subprocess.run(["kaggle", "datasets", "download", "-d", dataset_slug, "-p", str(target_path.parent), "--unzip"], check=True)
            print(f"✅ {dataset_slug} baixado e extraído para {target_path.parent}!")
            
            # O Kaggle pode baixar para uma pasta com o nome do slug, vamos renomear se necessário
            # e garantir que a estrutura seja a esperada.
            downloaded_dir = target_path.parent / dataset_slug.split('/')[-1]
            if downloaded_dir.exists() and downloaded_dir != target_path:
                print(f"Renomeando '{downloaded_dir}' para '{target_path}'...")
                shutil.move(downloaded_dir, target_path)
            
        except subprocess.CalledProcessError as e:
            print(f"❌ Erro ao baixar {dataset_slug} do Kaggle: {e}")
            print("Por favor, verifique se suas credenciais do Kaggle estão corretas e se você tem permissão para baixar o dataset.")
            print("Você pode precisar aceitar as regras do dataset no Kaggle primeiro: https://www.kaggle.com/datasets/hasyimabdillah/workoutexercises-images/code")
            return False
        except Exception as e:
            print(f"❌ Erro inesperado ao baixar {dataset_slug}: {e}")
            return False
    else:
        print(f"ℹ️ Dataset '{DATASET_NAME}' já existe em {target_path}. Pulando download.")
    return True

# Baixar o dataset de exercícios
dataset_downloaded = download_exercise_dataset(DATASET_SLUG, DATASET_PATH)

if not dataset_downloaded:
    print("❌ Não foi possível continuar sem o dataset. Encerrando o script.")
    sys.exit(1)


# Listar os tipos de exercícios (subpastas dentro do dataset)
exercise_types = [d.name for d in DATASET_PATH.iterdir() if d.is_dir()]
print(f"\n✅ Tipos de exercícios encontrados no dataset: {exercise_types}")

# Processamento dos Dados de Exercícios: Extrair Features e Gerar Rótulos de Forma (Correta/Incorreta)

# Listas para armazenar as features e os rótulos
all_features = []
all_exercise_labels = []
all_form_labels = [] # 0 para correto, 1 para incorreto

# Inicializar o objeto MediaPipe Pose para usar na extração de features
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_detector:
    for exercise_type in exercise_types:
        exercise_dir = DATASET_PATH / exercise_type
        image_files = [f for f in exercise_dir.iterdir() if f.suffix.lower() in ('.jpg', '.jpeg', '.png')]
        
        print(f"\n⚙️ Processando {len(image_files)} imagens para o exercício: {exercise_type}")
        
        # Amostragem para limitar o tamanho do dataset e acelerar o processamento durante o desenvolvimento.
        # Reduzir este número acelerará o tempo de pré-processamento e treinamento.
        SAMPLE_PER_EXERCISE = 400 # Processar até 400 imagens por tipo de exercício para um teste rápido
        
        if len(image_files) > SAMPLE_PER_EXERCISE:
            # Use list() para garantir que a amostra seja uma lista de caminhos, não um array numpy
            image_files_sampled = np.random.choice(image_files, SAMPLE_PER_EXERCISE, replace=False).tolist()
            print(f"   Amostrando {SAMPLE_PER_EXERCISE} imagens de {len(image_files)} disponíveis para {exercise_type}.")
        else:
            image_files_sampled = image_files.tolist() # Se menos que a amostra, usar todas.

        # Definir a proporção de exemplos "incorretos" que queremos gerar sinteticamente
        # Isso ajuda a balancear o dataset para as classes "correta" e "incorreta"
        synthetic_incorrect_ratio = 0.4 # Queremos que 40% das amostras processadas sejam sinteticamente incorretas
        num_samples_to_make_incorrect = int(len(image_files_sampled) * synthetic_incorrect_ratio)
        
        # Selecionar aleatoriamente os índices das imagens onde aplicaremos o erro sintético
        # Certifique-se de que não tentamos selecionar mais índices do que amostras disponíveis
        indices_to_make_incorrect = np.random.choice(
            len(image_files_sampled),
            min(num_samples_to_make_incorrect, len(image_files_sampled)),
            replace=False
        )
        
        for i, img_file in enumerate(tqdm(image_files_sampled, desc=f"   Extraindo features de {exercise_type}")):
            original_features = extract_pose_features(str(img_file), pose_detector)
            
            if original_features is not None:
                # Decide if this sample should be made synthetically incorrect
                if i in indices_to_make_incorrect:
                    # Apply synthetic error and label as incorrect
                    features_to_add = synthesize_incorrect_form(original_features, exercise_type)
                    form_status = 1 # Incorreto
                else:
                    # Keep original features and label as correct
                    features_to_add = original_features
                    form_status = 0 # Correto

                all_features.append(features_to_add)
                all_exercise_labels.append(exercise_type)
                all_form_labels.append(form_status)
            else:
                pass # Nenhuma pose detectada ou erro de carregamento de imagem


# Verificar se há dados para processar
if not all_features:
    print("\n❌ Nenhuma feature foi extraída. Verifique seu dataset e a detecção de pose.")
    print("O script será encerrado pois não há dados para treinar o modelo.")
    sys.exit(1) # Encerrar o script

# Converter listas para arrays NumPy
X = np.array(all_features)
y_exercise_raw = np.array(all_exercise_labels)
y_form = np.array(all_form_labels)

# Codificar rótulos de exercício e forma para formato numérico (one-hot encoding)
# Para exercícios, precisamos de um LabelEncoder primeiro para mapear strings para inteiros.
exercise_encoder = LabelEncoder()
y_exercise_encoded = exercise_encoder.fit_transform(y_exercise_raw)
y_exercise = to_categorical(y_exercise_encoded)

# Para a forma, já temos 0/1, então podemos diretamente para one-hot se necessário.
# Como é binário (0/1), to_categorical é útil para a função de perda.
y_form = to_categorical(y_form, num_classes=2)

print(f"\n✅ Total de {len(X)} amostras processadas.")
print(f"   Shape das features (X): {X.shape}")
print(f"   Shape dos rótulos de exercício (y_exercise): {y_exercise.shape}")
print(f"   Shape dos rótulos de forma (y_form): {y_form.shape}")
print(f"   Exercícios detectados: {exercise_encoder.classes_}")

# Criar diretório para salvar modelos e scalers
MODELS_DIR = Path('models')
MODELS_DIR.mkdir(exist_ok=True) # Garantir que a pasta models existe

# Salvar o LabelEncoder para uso posterior na inferência
joblib.dump(exercise_encoder, MODELS_DIR / 'exercise_encoder.pkl')
print(f"✅ exercise_encoder salvo em {MODELS_DIR / 'exercise_encoder.pkl'}")

# Normalizar as features (importante para redes neurais)
# Usaremos MinMaxScaler para escalar as features entre 0 e 1.
feature_scaler = MinMaxScaler()
X_scaled = feature_scaler.fit_transform(X)
joblib.dump(feature_scaler, MODELS_DIR / 'feature_scaler.pkl') # Salvar o feature_scaler
print(f"✅ feature_scaler salvo em {MODELS_DIR / 'feature_scaler.pkl'}")

print("✅ Pré-processamento de dados de exercícios concluído e escalers salvos.")

# 5. Construção e Treinamento do Modelo de Detecção de Forma

# Função para construir e compilar um modelo Keras (MLP)
def build_keras_mlp_model(input_shape, num_exercise_classes):
    input_layer = Input(shape=(input_shape,))
    x = Dense(512, activation='relu')(input_layer)
    x = BatchNormalization()(x) # Ajuda na estabilidade do treinamento
    x = Dropout(0.4)(x) # Previne overfitting
    
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # Duas saídas para as duas tarefas de classificação
    exercise_output = Dense(num_exercise_classes, activation='softmax', name='exercise_output')(x)
    form_output = Dense(2, activation='softmax', name='form_output')(x) # 2 classes: correto/incorreto
    
    model = Model(inputs=input_layer, outputs=[exercise_output, form_output])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'exercise_output': 'categorical_crossentropy',
            'form_output': 'categorical_crossentropy'
        },
        loss_weights={
            'exercise_output': 0.7, # Maior peso para classificar o exercício corretamente
            'form_output': 0.3 # Menor peso, mas ainda importante para a forma
        },
        metrics={
            'exercise_output': ['accuracy'],
            'form_output': ['accuracy']
        }
    )
    return model

# Função para treinar e salvar modelos Scikit-learn (Random Forest, Logistic Regression)
# Estes modelos predizem apenas o tipo de exercício
def train_and_save_sklearn_model(model_class, model_name, X_train, y_exercise_train_flat, X_val, y_exercise_val_flat):
    print(f"⚙️ Treinando modelo Scikit-learn: {model_name}")
    # Algumas classes de modelo podem precisar de hiperparâmetros específicos
    if model_name == "Logistic Regression":
        model = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
    elif model_name == "Random Forest":
        model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    else: # Fallback para outros, embora tenhamos apenas RF e LR por enquanto
        model = model_class(random_state=42)
        
    model.fit(X_train, y_exercise_train_flat)
    
    # Avaliar no conjunto de validação
    y_pred_val = model.predict(X_val)
    val_accuracy = accuracy_score(y_exercise_val_flat, y_pred_val)
    print(f"   Acurácia de Validação para {model_name}: {val_accuracy:.4f}")
    
    # Construção da string do nome do arquivo separadamente para evitar conflito de aspas
    model_filename_suffix = f'best_{model_name.lower().replace(" ", "_")}_model.pkl'
    model_filepath = MODELS_DIR / model_filename_suffix

    joblib.dump(model, model_filepath) # Usa o Path construído
    print(f"✅ Modelo '{model_name}' treinado e salvo em {model_filepath}") # Usa o Path construído
    return model, val_accuracy

# Dividir os dados em conjuntos de treino e validação
X_train, X_val, y_exercise_train, y_exercise_val, y_form_train, y_form_val = train_test_split(
    X_scaled, y_exercise, y_form, test_size=0.2, random_state=42
)

# Converter y_exercise_train/val de one-hot para rótulos flat para modelos Scikit-learn
y_exercise_train_flat = np.argmax(y_exercise_train, axis=1)
y_exercise_val_flat = np.argmax(y_exercise_val, axis=1)

num_features = X_scaled.shape[1]
num_exercise_classes = y_exercise.shape[1]

# Dicionário para armazenar o desempenho de cada modelo
model_performance = {}

# --- Treinar e Avaliar o Modelo Keras MLP ---
mlp_model_path = MODELS_DIR / 'best_mlp_model.h5'
if os.path.exists(mlp_model_path):
    print(f"ℹ️ Modelo MLP pré-treinado encontrado. Carregando...")
    mlp_model = load_model(mlp_model_path)
    print(f"✅ Modelo MLP carregado.")
else:
    print(f"⚙️ Modelo MLP não encontrado. Iniciando construção e treinamento...")
    mlp_model = build_keras_mlp_model(num_features, num_exercise_classes)
    
    checkpoint_mlp = ModelCheckpoint(
        mlp_model_path,
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    )
    reduce_lr_mlp = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
    early_stop_mlp = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )
    
    print(f"Iniciando treinamento para o modelo MLP...")
    history_mlp = mlp_model.fit(
        X_train,
        {'exercise_output': y_exercise_train, 'form_output': y_form_train},
        epochs=100000,
        batch_size=1080,
        validation_data=(X_val, {'exercise_output': y_exercise_val, 'form_output': y_form_val}),
        callbacks=[checkpoint_mlp, reduce_lr_mlp, early_stop_mlp],
        verbose=1
    )
    print(f"✅ Modelo MLP treinado e salvo em {mlp_model_path}")

# Avaliar o modelo MLP no conjunto de validação
print(f"Avaliando o modelo MLP no conjunto de validação...")
val_results_mlp = mlp_model.evaluate(X_val, {'exercise_output': y_exercise_val, 'form_output': y_form_val}, verbose=0)
val_loss_mlp = val_results_mlp[0]
val_exercise_accuracy_mlp = val_results_mlp[3] # Índice 3 para accuracy da exercise_output
val_form_accuracy_mlp = val_results_mlp[4] # Índice 4 para accuracy da form_output
model_performance['MLP Keras (Exercício & Forma)'] = {
    'val_loss': val_loss_mlp,
    'val_exercise_accuracy': val_exercise_accuracy_mlp,
    'val_form_accuracy': val_form_accuracy_mlp,
    'type': 'keras_multi_output',
    'path': mlp_model_path
}
print(f"MLP Keras - Perda de Validação: {val_loss_mlp:.4f}, Acurácia Exercício de Validação: {val_exercise_accuracy_mlp:.4f}, Acurácia Forma de Validação: {val_form_accuracy_mlp:.4f}")


# --- Treinar e Avaliar Modelos Scikit-learn ---
# Random Forest
rf_model, rf_val_accuracy = train_and_save_sklearn_model(RandomForestClassifier, "Random Forest", X_train, y_exercise_train_flat, X_val, y_exercise_val_flat)
model_performance['Random Forest (Apenas Exercício)'] = {
    'val_exercise_accuracy': rf_val_accuracy,
    'type': 'sklearn_single_output',
    'path': MODELS_DIR / f'best_random_forest_model.pkl' # Este já estava ok
}

# Logistic Regression
lr_model, lr_val_accuracy = train_and_save_sklearn_model(LogisticRegression, "Regressão Logística", X_train, y_exercise_train_flat, X_val, y_exercise_val_flat)
model_performance['Regressão Logística (Apenas Exercício)'] = {
    'val_exercise_accuracy': lr_val_accuracy,
    'type': 'sklearn_single_output',
    'path': MODELS_DIR / f'best_logistic_regression_model.pkl' # Este já estava ok
}


# --- Selecionar o Modelo Mais Eficiente ---
best_model_for_exercise_classification_name = None
highest_exercise_accuracy = -1

print("\n--- Resultados da Comparação de Modelos (Validação) ---")
for name, metrics in model_performance.items():
    print(f"Modelo: {name}")
    if 'val_loss' in metrics:
        print(f"  Perda de Validação: {metrics['val_loss']:.4f}")
    print(f"  Acurácia de Exercício de Validação: {metrics['val_exercise_accuracy']:.4f}")
    if 'val_form_accuracy' in metrics:
        print(f"  Acurácia de Forma de Validação: {metrics['val_form_accuracy']:.4f}")
    
    # Para a comparação de qual modelo é "mais eficiente" para o propósito do aplicativo,
    # o modelo Keras MLP é o único que oferece detecção de forma.
    # No entanto, para fins de relatório de comparação de acurácia de exercício,
    # identificamos o de maior acurácia de exercício.
    if metrics['val_exercise_accuracy'] > highest_exercise_accuracy:
        highest_exercise_accuracy = metrics['val_exercise_accuracy']
        best_model_for_exercise_classification_name = name

print(f"\n🏆 Modelo com maior acurácia para Classificação de Exercício (com base na acurácia de validação): '{best_model_for_exercise_classification_name}' com acurácia: {highest_exercise_accuracy:.4f}")

# Para a aplicação em tempo real, o modelo Keras MLP é escolhido porque é o único
# que fornece ambas as detecções: tipo de exercício E forma.
final_model_to_use_name = "MLP Keras (Exercício & Forma)"
final_model_to_use_path = model_performance[final_model_to_use_name]['path']
final_model_type = model_performance[final_model_to_use_name]['type']

print(f"\nEscolha final para aplicação em tempo real (devido à saída combinada de exercício e forma): '{final_model_to_use_name}'")
print(f"Carregando este modelo para detecção em tempo real: {final_model_to_use_path}")

# Carregar o modelo selecionado final para detecção em tempo real
if final_model_type == 'keras_multi_output':
    model = load_model(final_model_to_use_path)
    # Garantir que exercise_encoder e feature_scaler estejam carregados se não estiverem na memória
    if 'exercise_encoder' not in locals() or 'feature_scaler' not in locals():
        exercise_encoder = joblib.load(MODELS_DIR / 'exercise_encoder.pkl')
        feature_scaler = joblib.load(MODELS_DIR / 'feature_scaler.pkl')
else:
    # Este caso não deve ser alcançado se o MLP Keras for sempre escolhido para tempo real
    # mas incluído para completude caso a lógica mude.
    model = joblib.load(final_model_to_use_path)
    if 'exercise_encoder' not in locals() or 'feature_scaler' not in locals():
        exercise_encoder = joblib.load(MODELS_DIR / 'exercise_encoder.pkl')
        feature_scaler = joblib.load(MODELS_DIR / 'feature_scaler.pkl')
    print("AVISO: Um modelo não-Keras multi-saída foi selecionado para tempo real. O feedback de forma pode ser limitado.")


# 6. Detecção de Forma de Exercícios em Tempo Real

# Mapeamento para os rótulos de forma
form_labels = ['Forma Correta', 'Forma Incorreta']

# Inicializar o MediaPipe Pose para detecção em tempo real
# 'static_image_mode=False' para melhor desempenho em streams de vídeo.
# O MediaPipe automaticamente usa a CPU neste ambiente.
pose_live_detector = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Função de predição em tempo real
def detect_exercise_form(frame, pose_detector, model, feature_scaler, exercise_encoder, form_labels, model_type):
    """
    Processa um frame da webcam para detectar a pose e prever o exercício e a forma.
    Args:
        frame (numpy.ndarray): O frame de vídeo da webcam.
        pose_detector (mp.solutions.pose.Pose): Objeto MediaPipe Pose inicializado (executando em CPU).
        model (tensorflow.keras.Model): O modelo de ML treinado (inferência em CPU).
        feature_scaler (MinMaxScaler): O scaler usado para normalizar as features.
        exercise_encoder (LabelEncoder): O encoder usado para os rótulos de exercício.
        form_labels (list): Lista de rótulos para a forma (e.g., ['Correto', 'Incorreto']).
        model_type (str): Tipo do modelo (e.g., 'keras_multi_output', 'sklearn_single_output').
    Returns:
        numpy.ndarray: O frame com as informações de pose e feedback.
    """
    # Converter o frame para RGB para o MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Melhorar o desempenho (opcional): marcar a imagem como não gravável para passar por referência.
    image_rgb.flags.writeable = False
    
    # Processar o frame para estimativa de pose (executa na CPU)
    results = pose_detector.process(image_rgb)
    
    # Marcar a imagem como gravável novamente para desenhar as anotações
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR) # Voltar para BGR para o OpenCV
    
    feedback_text = "Nenhuma pose detectada."
    text_color = (255, 255, 255) # Cor padrão branca para feedback

    if results.pose_landmarks:
        # Desenhar as landmarks da pose no frame
        mp_drawing.draw_landmarks(
            image_bgr,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing_styles.get_default_pose_landmarks_style()
        )
        
        # Extrair features da pose detectada no frame atual
        # Replicamos a lógica de extract_pose_features para o frame atual
        landmarks = results.pose_landmarks.landmark
        
        try:
            # Coletar as mesmas features que foram usadas no treinamento
            live_features = np.array([
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                )
            ])
            
            # Normalizar as features usando o scaler treinado
            live_features_scaled = feature_scaler.transform(live_features.reshape(1, -1))
            
            if model_type == 'keras_multi_output':
                predictions = model.predict(live_features_scaled, verbose=0)
                exercise_probs = predictions[0][0]
                form_probs = predictions[1][0]
                
                predicted_exercise_idx = np.argmax(exercise_probs)
                predicted_exercise = exercise_encoder.inverse_transform([predicted_exercise_idx])[0]
                
                predicted_form_idx = np.argmax(form_probs)
                predicted_form = form_labels[predicted_form_idx]
                
                feedback_text = f"Exercicio: {predicted_exercise} ({exercise_probs[predicted_exercise_idx]*100:.1f}%) | Forma: {predicted_form} ({form_probs[predicted_form_idx]*100:.1f}%)"
                
                if predicted_form_idx == 1: # Se a forma for incorreta
                    text_color = (0, 0, 255) # Vermelho
                    feedback_text += " -> Ajuste sua postura!"
                else:
                    text_color = (0, 255, 0) # Verde para correto
            else: # Modelos Scikit-learn (ou qualquer outro modelo de saída única para exercício)
                # Estes modelos predizem apenas o tipo de exercício.
                # Assumindo que os modelos sklearn foram selecionados por sua maior acurácia de exercício.
                # Para a 'forma', vamos assumir 'Correta' por padrão, já que eles não a preveem explicitamente.
                exercise_pred_id = model.predict(live_features_scaled)[0]
                exercise_probs_all = model.predict_proba(live_features_scaled)[0] # Obter probabilidades para todas as classes
                
                predicted_exercise = exercise_encoder.inverse_transform([exercise_pred_id])[0]
                confidence = exercise_probs_all[exercise_pred_id]
                
                # Forma padrão para correta para modelos de saída única, pois eles não preveem a forma explicitamente
                predicted_form = "Forma Correta" 
                predicted_form_idx = 0 # Padrão para índice de forma correta
                form_probs = np.array([1.0, 0.0]) # Assumir 100% de confiança na forma correta para fins de exibição
                
                feedback_text = f"Exercicio: {predicted_exercise} ({confidence*100:.1f}%) | Forma: {predicted_form}"
                text_color = (0, 255, 0) # Sempre verde se a forma for assumida como correta

        except Exception as e:
            feedback_text = f"Erro na predição: {e}"
            text_color = (0, 165, 255) # Laranja para erro
            print(f"Erro na detecção: {e}")

    # Exibir o feedback no frame
    cv2.putText(image_bgr, feedback_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2, cv2.LINE_AA)
    
    return image_bgr

# Loop da Webcam para Detecção em Tempo Real

# Inicializar a captura de vídeo da webcam (0 é geralmente a câmera padrão)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Não foi possível acessar a webcam. Verifique se ela está conectada e não está em uso.")
else:
    # Definir resolução da webcam para melhor qualidade
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    window_name = 'Detecao de Forma de Exercicios (CPU) - Pressione Q para Sair'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    
    print("\n✅ Webcam ativada. Pressione 'Q' para sair da janela de visualização.")
    # Exibir a hora atual conforme as instruções
    current_utc_time = time.gmtime()
    current_brasilia_time = time.localtime(time.time() - 3 * 3600) # UTC-3
    print(f"UTC: {time.strftime('%d/%m/%Y %H:%M:%S', current_utc_time)} (UTC)")
    print(f"Brasília: {time.strftime('%d/%m/%Y %H:%M:%S', current_brasilia_time)} (UTC-3)")
    print("\n⚠️ A performance pode ser limitada pela sua CPU. Para melhor experiência, considere GPUs dedicadas.")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Falha ao ler frame da webcam.")
                break
                
            # Espelhar o frame para que a visualização seja mais intuitiva (como um espelho)
            frame = cv2.flip(frame, 1)
            
            # Chamar a função de detecção e obter o frame com feedback
            output_frame = detect_exercise_form(frame, pose_live_detector, model, feature_scaler, exercise_encoder, form_labels, final_model_type)
            
            # Mostrar o frame na janela
            cv2.imshow(window_name, output_frame)
            
            # Verificar se a janela foi fechada pelo usuário
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
                
            # Sair do loop se a tecla 'q' for pressionada
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    finally:
        # Liberar os recursos da webcam e fechar todas as janelas do OpenCV
        cap.release()
        cv2.destroyAllWindows()
        print("✅ Programa finalizado. Recursos liberados.")

ℹ️ Nenhuma GPU NVIDIA detectada ou configurada. Todas as operações de Machine Learning utilizarão a CPU.
✅ Ambiente configurado. Credenciais do Kaggle salvas e ambiente preparado para CPU.
✅ Funções de extração de pose, cálculo de ângulos e geração de erros sintéticos prontas.
ℹ️ Dataset 'workout-exercises-images' já existe em workout-exercises-images. Pulando download.

✅ Tipos de exercícios encontrados no dataset: ['barbell biceps curl', 'bench press', 'chest fly machine', 'deadlift', 'decline bench press', 'hammer curl', 'hip thrust', 'incline bench press', 'lat pulldown', 'lateral raises', 'leg extension', 'leg raises', 'plank', 'pull up', 'push up', 'romanian deadlift', 'russian twist', 'shoulder press', 'squat', 't bar row', 'tricep dips', 'tricep pushdown']

⚙️ Processando 705 imagens para o exercício: barbell biceps curl
   Amostrando 400 imagens de 705 disponíveis para barbell biceps curl.


   Extraindo features de barbell biceps curl: 100%|██████████████████████████████████| 400/400 [00:21<00:00, 18.81it/s]



⚙️ Processando 625 imagens para o exercício: bench press
   Amostrando 400 imagens de 625 disponíveis para bench press.


   Extraindo features de bench press: 100%|██████████████████████████████████████████| 400/400 [00:19<00:00, 20.69it/s]



⚙️ Processando 527 imagens para o exercício: chest fly machine
   Amostrando 400 imagens de 527 disponíveis para chest fly machine.


...

   Extraindo features de tricep dips: 100%|██████████████████████████████████████████| 400/400 [00:21<00:00, 18.41it/s]



⚙️ Processando 625 imagens para o exercício: tricep pushdown
   Amostrando 400 imagens de 625 disponíveis para tricep pushdown.


   Extraindo features de tricep pushdown: 100%|██████████████████████████████████████| 400/400 [00:21<00:00, 18.52it/s]



✅ Total de 7852 amostras processadas.
   Shape das features (X): (7852, 8)
   Shape dos rótulos de exercício (y_exercise): (7852, 22)
   Shape dos rótulos de forma (y_form): (7852, 2)
   Exercícios detectados: ['barbell biceps curl' 'bench press' 'chest fly machine' 'deadlift'
 'decline bench press' 'hammer curl' 'hip thrust' 'incline bench press'
 'lat pulldown' 'lateral raises' 'leg extension' 'leg raises' 'plank'
 'pull up' 'push up' 'romanian deadlift' 'russian twist' 'shoulder press'
 'squat' 't bar row' 'tricep dips' 'tricep pushdown']
✅ exercise_encoder salvo em models\exercise_encoder.pkl
✅ feature_scaler salvo em models\feature_scaler.pkl
✅ Pré-processamento de dados de exercícios concluído e escalers salvos.
ℹ️ Modelo MLP pré-treinado encontrado. Carregando...
✅ Modelo MLP carregado.
Avaliando o modelo MLP no conjunto de validação...
MLP Keras - Perda de Validação: 1.1351, Acurácia Exercício de Validação: 0.5977, Acurácia Forma de Validação: 0.6155
⚙️ Treinando modelo Scikit

   Acurácia de Validação para Regressão Logística: 0.3393
✅ Modelo 'Regressão Logística' treinado e salvo em models\best_regressão_logística_model.pkl

--- Resultados da Comparação de Modelos (Validação) ---
Modelo: MLP Keras (Exercício & Forma)
  Perda de Validação: 1.1351
  Acurácia de Exercício de Validação: 0.5977
  Acurácia de Forma de Validação: 0.6155
Modelo: Random Forest (Apenas Exercício)
  Acurácia de Exercício de Validação: 0.6595
Modelo: Regressão Logística (Apenas Exercício)
  Acurácia de Exercício de Validação: 0.3393

🏆 Modelo com maior acurácia para Classificação de Exercício (com base na acurácia de validação): 'Random Forest (Apenas Exercício)' com acurácia: 0.6595

Escolha final para aplicação em tempo real (devido à saída combinada de exercício e forma): 'MLP Keras (Exercício & Forma)'
Carregando este modelo para detecção em tempo real: models\best_mlp_model.h5

✅ Webcam ativada. Pressione 'Q' para sair da janela de visualização.
UTC: 09/07/2025 03:05:03 (UTC)
Bras